In [20]:
! pip install anthropic numpy scipy matplotlib

In [21]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [22]:
import anthropic
import numpy as np
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

In [33]:
from voyageai import Client
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
!pip install -U voyageai

In [34]:
client = Client(api_key="pa-hcwX_-1DRDb_5vmEThd36CnfwI5woxSGLWCtravhRZQ")

In [26]:
df = pd.read_csv('상담대화_평가용.csv')

In [27]:
df.head()

,Unnamed: 0,speaker,text
0,0,1,안녕하세요. 상담사입니다. 무엇이 불편하시나요?
1,1,0,내가 약간 중2병 같은 걸 증상을 보이고 있어요.
2,2,1,"중2병 증상이라니, 어떤 증상이신 건가요?"
3,3,0,"그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런..."
4,4,1,그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?


In [29]:
len(df)

135016

## 일단은 우리 output을 speaker ==0, 기존 output을 speaker ==1로 두고 진행

In [30]:
df['cosine_similarity'] = np.nan

In [31]:
df

,Unnamed: 0,speaker,text,cosine_similarity
0,0,1,안녕하세요. 상담사입니다. 무엇이 불편하시나요?,NaN
1,1,0,내가 약간 중2병 같은 걸 증상을 보이고 있어요.,NaN
2,2,1,"중2병 증상이라니, 어떤 증상이신 건가요?",NaN
3,3,0,"그러니까 공부하기 싫어하고, 공격적이고, 좀 무례하게 말하고 싶은 게 많아져서 그런...",NaN
4,4,1,그런 증상이 있으니까 힘드시겠죠. 중2병 같은 것이라고 생각하시는 이유는 무엇인가요?,NaN
...,...,...,...,...
135011,135011,0,그러면 자기 존중감을 다시 찾는 것은 어떻게 해야 하나요?,NaN
135012,135012,1,자기 존중감을 회복하기 위해서는 자신의 업무 능력에 대한 자신감을 회복하시는 것이 ...,NaN
135013,135013,0,감사합니다. 저는 금방 내일 일찍 일어나서 운동하고 오늘하루 정리하는 습관을 만들고...,NaN
135014,135014,1,그것도 좋은 생각이에요. 스트레스를 완화하기 위해서는 헬스장에서 운동을 하거나 스트...,NaN


In [42]:
def get_similarity(text1, text2):
  embedding1 = client.embed(text1).embeddings
  embedding2 = client.embed(text2).embeddings

  embedding1_np = np.array(embedding1).flatten()
  embedding2_np = np.array(embedding2).flatten()
  similarity = cosine_similarity([embedding1_np], [embedding2_np])[0][0]
  return similarity

In [44]:
for i in range(0, 299, 2):
  text1 = df.loc[i, 'text']
  text2 = df.loc[i+1, 'text']

  similarity = get_similarity(text1, text2)
  df.loc[i+1, 'cosine_similarity'] =similarity

/usr/local/lib/python3.10/dist-packages/voyageai/client.py:61: UserWarning: The `model` argument is not specified and defaults to voyage-2. It will be a required argument in the future. We recommend to specify the model when using this function. Please see https://docs.voyageai.com/docs/embeddings for the list of latest models provided by Voyage AI.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/voyageai/client.py:61: UserWarning: The `model` argument is not specified and defaults to voyage-2. It will be a required argument in the future. We recommend to specify the model when using this function. Please see https://docs.voyageai.com/docs/embeddings for the list of latest models provided by Voyage AI.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/voyageai/client.py:61: UserWarning: The `model` argument is not specified and defaults to voyage-2. It will be a required argument in the future. We recommend to specify the model when using this function. Please see https

In [52]:
mean_similarity = df['cosine_similarity'].dropna().mean()
print(mean_similarity)

0.8247064219489946


In [15]:
#from sklearn.decomposition import PCA

#pca = PCA(n_components=3)
#embeddings_3d = pca.fit_transform(embeddings)

In [54]:
#3d plot

#plt.rc('font', family='NanumBarunGothic')
#fig = plt.figure(figsize = (12,8))
#ax = fig.add_subplot(111, projection='3d')

#for i, (x,y,z) in enumerate(embeddings_3d):
#  ax.scatter(x,y,z)
#  ax.text(x,y,z, f'문장 {i+1}')


#ax.set_xlabel('X')
#ax.set_ylabel('Y')
#ax.set_zlabel('Z')
#ax.set_title('문장 임베딩의 3차원 표현')

#plt.show()